In [7]:
import os
import skimage as ski
import watershed_udder as wu
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyrealsense2 as rs
from skimage.restoration import inpaint
import shapely
from skimage.measure import find_contours
from shapely import LineString, MultiPoint, Polygon
from skimage.transform import rotate
from astropy.convolution import Gaussian2DKernel, convolve,interpolate_replace_nans

In [3]:
def get_angle(right_kp, left_kp):
    angle = np.arctan2(right_kp[1]-left_kp[1], right_kp[0]-left_kp[0])
    return angle

def get_center(right_kp, left_kp):
    return shapely.centroid(MultiPoint([right_kp, left_kp])).coords[0] 

def get_orientation(right_kp, left_kp):
    if right_kp[0] < left_kp[0]:
        orientation = -1 # up 
    else: 
        orientation = 1 # down
    return orientation

def rotate_udder(udder, right_kp, left_kp):
    k = get_orientation(right_kp, left_kp)
    center = get_center(right_kp, left_kp)
    angle = get_angle(right_kp, left_kp)
    rotated_udder = rotate(udder, np.rad2deg(k*angle), center = center, preserve_range = True)
    return rotated_udder

def rotate_points(right_kp, left_kp):
    k = get_orientation(right_kp, left_kp)
    points = np.concatenate([[right_kp], [left_kp]])
    points2 = points.copy()
    angle = get_angle(right_kp, left_kp)
    center = get_center(right_kp, left_kp)
    rot_mat = np.array([[np.cos(-k*angle), -np.sin(-k*angle)], [np.sin(-k*angle), np.cos(-k*angle)]])
    #
    points2[:, 0] = points[:, 0] - center[0]
    points2[:, 1] = points[:, 1] - center[1]
    # 
    points2 = np.transpose(np.dot(rot_mat, np.transpose(points2[:, :2])))
    points2[:, 0] = points2[:, 0] + center[0]
    points2[:, 1] = points2[:, 1] + center[1]
    rotated_points = points2.copy()
    
    return rotated_points


def udder_line(udder_object, udder_shp, rf_kp, lf_kp):
    img = udder_object.img
    im_width =udder_object.size[1]
    udder2 = rotate_udder(img, rf_kp, lf_kp)
    points2 = rotate_points(rf_kp, lf_kp)
    yloc = np.floor(points2[0,1]).astype(int)
    # fig, ax = plt.subplots()
    for i in range(-10,10):
        yloc2 = yloc + i 
        line = LineString([(0, yloc2), (im_width, yloc2)])
        intersection = udder_shp.exterior.intersection(line).geoms
        endpoints = np.array([list(intersection[0].coords[0]), list(intersection[1].coords[0])])
        start = np.floor(endpoints[np.argmin(endpoints[:, 0])]).astype(int)
        end = np.floor(endpoints[np.argmax(endpoints[:, 0])]).astype(int)
        line_vals = udder2[yloc2][list(range(start[0], end[0]))]
        plt.plot(line_vals)

In [8]:
def update_kp(kp_ws, ws_label, img):
    newkp_dict = {}
    # fig, axs = plt.subplots(ncols = 4, nrows= 1, figsize = (12, 4))
    for key in kp_ws.keys():
        label = kp_ws[key]
        mask = ws_label.copy()
        mask[mask!= label] = 0
        mask[mask == label] = 1
        quarter = (mask*img).astype(float)
        quarter[quarter==0] =np.nan
        mins = np.argwhere(quarter== np.nanmin(quarter))
        x = np.round(np.median(mins[:, 1]), 0).astype(int)
        y = np.round(np.median(mins[:, 0]), 0).astype(int)
        newkp_dict[key] = (x,y)
    
    return newkp_dict

In [9]:
# list files 
dirpath = os.getcwd()
ws_dir = r"validate_watershed\watershed_segments"
corr_dir = r"validate_watershed\watershed_correspondence"
label_dir = os.path.join(dirpath, r"validate_watershed\pred_labels")
kp_dir = os.path.join(label_dir, r"keypoints")
sg_dir = os.path.join(label_dir, r"segments")
img_dir = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r"udder_video\depth_images")
filenames = [file.replace(".npy", "") for file in os.listdir(ws_dir)]

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'validate_watershed\\watershed_segments'

In [5]:
video_path =  os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r"udder_video\video_files\video_example.bag")

config = rs.config()
rs.config.enable_device_from_file(config, video_path, repeat_playback = False)
pipeline = rs.pipeline()
cfg = pipeline.start(config) # Start pipeline and get the configuration it found
profile = cfg.get_stream(rs.stream.depth) # Fetch stream profile for depth stream
intr = profile.as_video_stream_profile().get_intrinsics() # Downcast to video_stream_profile and fetch intrinsics

NameError: name 'dirpath' is not defined

In [ ]:
for file in filenames[:1]:
    # udder object
    udder = wu.udder_object(file + ".tif", img_dir, label_dir, array = 0)
    # read image
    img = udder.img
    # read labels
    segment = udder.get_segment()
    points = udder.get_keypoints()
    
    # reas WS segmentation
    ws_label = np.load(os.path.join(ws_dir, file + ".npy"))
    kp_ws = pd.read_csv(os.path.join(corr_dir, file +".csv")).loc[0].to_dict()

    new_kp = update_kp(kp_ws, ws_label, img)
    plt.imshow(img*udder.get_mask())
    plt.plot(new_kp["lf"][0], new_kp["lf"][1], "*r")
    plt.plot(new_kp["rf"][0], new_kp["rf"][1], "*b")
    plt.plot(new_kp["lb"][0], new_kp["lb"][1], "*r")
    plt.plot(new_kp["rb"][0], new_kp["rb"][1], "*b")
    plt.show()

In [97]:
def udder_line(udder_object, udder_shp, rf_kp, lf_kp):
    img = udder_object.img.copy().astype(float)
    im_width =udder_object.size[1]
    img[img ==0] = np.nan
    kernel = Gaussian2DKernel(x_stddev=1)
    udder_conv = convolve(img, kernel)
    udder2 = rotate_udder(udder_conv, rf_kp, lf_kp)
    points2 = rotate_points(rf_kp, lf_kp)
    yloc = np.floor(points2[0,1]).astype(int)
    # fig, ax = plt.subplots()
    # for i in range(0,1):
    yloc2 = yloc #  + i 
    line = LineString([(0, yloc2), (im_width, yloc2)])
    intersection = udder_shp.exterior.intersection(line).geoms
    endpoints = np.array([list(intersection[0].coords[0]), list(intersection[1].coords[0])])
    start = np.floor(endpoints[np.argmin(endpoints[:, 0])]).astype(int)
    end = np.floor(endpoints[np.argmax(endpoints[:, 0])]).astype(int)
    line_vals = udder2[yloc2][list(range(start[0], end[0]))]
    x = np.array(list(range(start[0],  end[0])))
    y = np.array([yloc]*len(x))
    z = line_vals
    return np.column_stack((x, y, z))

In [109]:
# derotate points 
def derotate_points(right_kp, left_kp, rotated_points):
    k = get_orientation(right_kp, left_kp)
    angle = -get_angle(right_kp, left_kp)
    center = get_center(right_kp, left_kp)
    rot_mat = np.array([[np.cos(-k*angle), -np.sin(-k*angle)], [np.sin(-k*angle), np.cos(-k*angle)]])
    points = rotated_points.copy()
    points[:, 0] = rotated_points[:, 0] - center[0]
    points[:, 1] = rotated_points[:, 1] - center[1]
    
    points = np.transpose(np.dot(rot_mat, np.transpose(points[:, :2])))
    
    points[:, 0] = points[:, 0] + center[0]
    points[:, 1] = points[:, 1] + center[1]
    
    derotated_points = np.floor(points).astype(int)
    return derotated_points

In [133]:
udder_shp = udder.get_shape()
line_vals = udder_line(udder, udder_shp, new_kp["rf"], new_kp["lf"])
line_vals_xy = derotate_points(new_kp["rf"], new_kp["lf"], line_vals[:, :2])
line_vals2 = np.column_stack((line_vals_xy[:, 0],line_vals_xy[:, 1], line_vals[:, 2]))
line_vals2 = line_vals2[~np.isnan(line_vals2[:, 2])]

line_vals2[:, 2] = line_vals2[:, 2]*scale

In [135]:
rs.rs2_deproject_pixel_to_point(intr, [line_vals2[0, 0], line_vals2[0, 1]], line_vals2[0, 2])

[-0.09610024839639664, 0.12390340864658356, 0.6283929944038391]

In [136]:
rs.rs2_deproject_pixel_to_point(intr, [line_vals2[1, 0], line_vals2[1, 1]], line_vals2[1, 2])

[-0.09468232095241547, 0.12393789738416672, 0.6285679340362549]

In [ ]:
rs.rs2_deproject_pixel_to_point(depth_intrin, [color_point[0], color_point[1]], depth